In [1]:
!pip install -U transformers datasets peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from datasets import load_dataset

dataset = load_dataset("commonsense_qa")
train_data, eval_data = dataset["train"], dataset["validation"]

print(train_data[0])


{'id': '075e483d21c29a511267ef62bedc0461', 'question': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?', 'question_concept': 'punishing', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']}, 'answerKey': 'A'}


In [7]:
from transformers import AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Required since GPT2 doesn't have a pad token


In [8]:
import numpy as np
def format_prompt(example):
    question = example['question']
    choices = example['choices']['text']
    labels = example['choices']['label']
    answer_key = example['answerKey']

    # Find the correct answer text
    answer_index = labels.index(answer_key)
    answer_text = choices[answer_index]

    # Create the prompt
    prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
    full_text = prompt + " " + answer_text

    # Tokenize full input
    tokenized = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    # Calculate where the completion starts
    prompt_ids = tokenizer(prompt,
                           truncation=True,
                           max_length=128)["input_ids"]
    completion_start = len(prompt_ids)

    # Mask prompt tokens in the labels
    labels = [-np.inf] * completion_start + tokenized["input_ids"][completion_start:]
    labels += [-np.inf] * (128 - len(labels))  # pad label to match input

    tokenized["labels"] = labels
    return tokenized


tokenized_train = train_data.map(format_prompt, batched=False, remove_columns=train_data.column_names)
tokenized_eval = eval_data.map(format_prompt, batched=False, remove_columns=eval_data.column_names)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [9]:
tokenized_train = train_data.map(format_prompt, batched=False, remove_columns=train_data.column_names)
tokenized_eval = eval_data.map(format_prompt, batched=False, remove_columns=eval_data.column_names)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

In [6]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.8 MB/s eta 0:00:00


In [1]:
!pip install --upgrade transformers accelerate

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
import torch

model_name = "gpt2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["c_attn", "c_proj"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


In [12]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


training_args = TrainingArguments(
    output_dir=f"./IA3_{model_name}",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",

    learning_rate=5e-3,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)


trainer.train()



No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,3.874200,3.279553
200,3.387000,3.230115
300,3.319000,3.193831
400,3.349300,3.165622
500,3.295800,3.169264
600,3.288900,3.176366
700,3.304200,3.115745
800,3.265700,3.118248
900,3.242900,3.118149
1000,3.243700,3.108700


TrainOutput(global_step=3654, training_loss=3.0239795461403065, metrics={'train_runtime': 483.0246, 'train_samples_per_second': 60.5, 'train_steps_per_second': 7.565, 'total_flos': 1927135929434112.0, 'train_loss': 3.0239795461403065, 'epoch': 3.0})

In [13]:
import math
from transformers import Trainer, TrainingArguments

eval_results = trainer.evaluate(eval_dataset=tokenized_eval)
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 14.45
